네이버 증권 종목 순위 크롤링 (https://kjk92.tistory.com/63 크롤링 라이브러리 설명)

In [ ]:
import pandas as pd

url = 'https://finance.naver.com/sise/lastsearch2.nhn'
dfs = pd.read_html(url,match='순위',encoding='euc-kr')

print(dfs[0][['순위', '종목명','현재가']].dropna(how="all"))

멜론 차트 크롤링 구글드라이브 활용 (https://velog.io/@rimi/Python-%EB%8B%A4%EB%93%A4-%ED%95%98%EB%8A%94-%ED%81%AC%EB%A1%A4%EB%A7%81-%EB%82%98%EB%8F%84-%ED%95%9C%EB%B2%88-%ED%95%B4%EB%B3%B4%EC%9E%90selenium%EB%B6%80%ED%84%B0-pandas%EA%B9%8C%EC%A7%80)

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from math import ceil
from itertools import chain
import pandas as pd
import time


options = webdriver.ChromeOptions()
options.add_argument('headless') 
options.add_argument('disable-gpu') 
options.add_argument('lang=ko_KR') 
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


def page_move(url):
    if isinstance(url, str):
        driver.get(url)
    else:
        driver.get(str(url))


def get_chart_list():
    try:
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, 'lst50')))
        chart_list = []
        data = driver.find_elements_by_class_name('lst50 > td')
        for k in data:
            chart_list.append(k.text.split('\n'))
        chain_list = list(chain(*chart_list))
        chuck_data = chunk(chain_list, 14)
        processing_chart_list(chuck_data)
    except TimeoutException:
        print('해당 페이지에 차트정보가 없습니다.')
    finally:
        driver.quit()


def processing_chart_list(chart_list):
    df = pd.DataFrame(chart_list, columns=['', '순위', '증감', '', '', '제목', '가수', '앨범', '좋아요', '좋아요수', '', '', '', ''])
    df.index = df.index + 1
    df.to_csv(f'chart_df.csv', mode='w', encoding='utf-8-sig', header=True, index=True)
    print(df)


def chunk(lst, size):
    return list(map(lambda x: lst[x * size:x * size + size], list(range(0, ceil(len(lst) / size)))))


if __name__ == '__main__':
    page_move('https://www.melon.com/chart/index.htm')
    get_chart_list()